In [1]:
import time
import pandas as pd
start_time = time.time()
# Sets
students = pd.read_csv('set_students.csv')['students'].tolist()
teachers = pd.read_csv('set_teachers.csv')['teachers'].tolist()
courses = pd.read_csv('set_courses.csv')['courses'].tolist()
timeslots = pd.read_csv('set_timeslots.csv')['timeslots'].tolist()
living_units = pd.read_csv('set_living_units.csv')['living Units'].tolist()

# Parameters
H = pd.read_csv('student_total_hours_without_varia.csv', sep=';').set_index('students').to_dict()['H_s']
m = pd.read_csv('student_courses_min_hours_without_varia.csv', sep=';').set_index(['students', 'courses']).to_dict()['min hours']
M = pd.read_csv('student_courses_max_hours_without_varia.csv', sep=';').set_index(['students', 'courses']).to_dict()['max hours']
V = pd.read_csv('student_courses_without_varia.csv', sep = ';').set_index('students').groupby('students')['courses'].apply(list).to_dict()
R = (pd.read_csv('student_teacher_relation.csv', sep=';')
    .pivot(index='Student', columns='teacher', values='relation')
    .fillna(0)
    .astype(int)
    .to_dict(orient='index')
)
T_avail_u = pd.read_csv('living_unit_hours.csv', sep=';').set_index('living units').groupby('living units')['timeslots'].apply(set).to_dict()
u_s = pd.read_csv('student_living_unit.csv', sep=';').set_index('students').groupby('students')['living units'].apply(set).to_dict()
T_unavail_s = pd.read_csv('student_unavailable.csv', sep=';').set_index('students').groupby('students')['unavailable'].apply(set).to_dict()
T_avail = {}

for student, living_unit_set in u_s.items():
    # Get the single living unit from the set
    living_unit = next(iter(living_unit_set), None)
    
    if not living_unit:
        T_avail[student] = set()
        continue

    # Timeslots available for the living unit
    lu_timeslots = T_avail_u.get(living_unit, set())

    # Timeslots the student is unavailable
    unavailable = T_unavail_s.get(student, set())

    # Subtract unavailable from living unit's available
    available = lu_timeslots - unavailable

    T_avail[student] = available

T_unavail_l = pd.read_csv('teacher_unavailable.csv', sep=';').set_index('teachers').groupby('teachers')['timeslots'].apply(set).to_dict()
c = pd.read_csv('teacher_course_competences.csv', sep=';').set_index(['teachers', 'course']).to_dict()['competence']
a = pd.read_csv('student_one_on_one.csv', sep=';').set_index('students').to_dict()['one-on-one']
x_prev = pd.read_csv('previous_week.csv', sep=';').set_index(['students', 'teachers', 'timeslots', 'courses']).to_dict()['assigned']# Model


for s in students:
    for l in teachers:
        for t in timeslots:
            for v in courses:
                if (s, l, t, v) not in x_prev:  # If the combination is not in the original x_prev
                    x_prev[(s, l, t, v)] = 0  # Assign a value of 0
        
# expand V[s] with imaginary students with no courses
for s in students:
    if s not in V:
        V[s] = []

  


In [2]:
#Phase 1 : Feasibility phase
def calculate_total_objective(x, students, teachers, timeslots, courses, V, R, H, m, M, T_avail, T_unavail_l, a):

    y = {}
    y_slv = {}
    # Loop over all students, teachers, timeslots, and courses to compute y_prev
    for s in students:
        for l in teachers:
            for t in timeslots:
                 #Sum over all courses v that student s is assigned to at timeslot t with teacher l
                y[(s, l, t)] = sum(x[s,l,t,v] for v in V[s])
    
    #compute y_slv
    for s in students:
        for l in teachers:      
            for v in V[s]:
                y_slv[s, l, v] = int(any(x[s, l, t, v] == 1 for t in timeslots))
                
    d_st_hc1 = {}
    d_s_hc2 = {}
    d_sv_hc5 = {}
    d_sv_hc6 = {}
    d_sltv_hc7 = {}
    d_slt_hc8 = {}
    d_sltv_hc10 = {}
    d_sltv_hc11 = {}
    d_sv_hc12 = {}


    
    #Assignment d_st_hc1:
    for s in students:
        for t in timeslots:
            total_classes = sum(x[s, l, t, v] for v in V[s] for l in teachers)
            d_st_hc1[s, t] = max(0, total_classes - 1)
        
    #Assignment d_s_hc2:
    for s in students:
        total_hours = sum(x[s, l, t, v] for v in V[s] for l in teachers for t in timeslots)
        d_s_hc2[s] = abs(total_hours - H[s])

    #Assignment d_sv_hc5:
    for s in students:
        for v in V[s]:
            d_sv_hc5[s, v] = abs(1 - sum(y_slv[s, l, v] for l in teachers))


    #Assignment d_sv_hc6:
    for s in students:
        for v in V[s]:
            total = sum(x[s, l, t, v] for l in teachers for t in timeslots)
            min_violation = max(0, m[s, v] - total)
            max_violation = max(0, total - M[s, v])
            d_sv_hc6[s, v] = min_violation + max_violation


    #Assignment d_sltv_hc7:
    for s in students:
        for l in teachers:
            for t in timeslots:
                for v in V[s]:
                    d_sltv_hc7[s, l, t, v] = x[s, l, t, v] * (1 - R[s][l])

    # Assignment d_slt_hc8: 
    for s in students:
        for l in teachers:
            for t in timeslots:
                sum_others = sum(x[s_prime, l, t, v_prime] for s_prime in students if s_prime != s for v_prime in V[s_prime])
                rhs = (1 - a[s]) + (1 - y[s, l, t]) * 2
                d_slt_hc8[s, l, t] = max(0, sum_others - rhs)


    #Assignment d_sltv_hc10:
    for s in students:
        for l in teachers:
            for t in timeslots:
                for v in V[s]:
                    if t not in T_avail[s]:
                        d_sltv_hc10[s,l,t,v] = x[s,l,t,v]
                    else:
                        d_sltv_hc10[s,l,t,v] = 0

    #Assignment d_sltv_hc11:                    
    for s in students:
        for l in teachers:
            for t in timeslots:
                for v in V[s]:
                    if t in T_unavail_l[l]:
                        d_sltv_hc11[s,l,t,v] = x[s,l,t,v]
                    else:
                        d_sltv_hc11[s,l,t,v] = 0

    #Assignment d_sv_hc12
    for s in students:
        for v in courses:
            if v not in V[s]:
                d_sv_hc12[s,v] = sum(x[s,l,t,v] for l in teachers for t in timeslots)

    total_objective = (
        sum(d_st_hc1.values()) +
        sum(d_s_hc2.values()) +
        sum(d_sv_hc5.values()) +
        sum(d_sv_hc6.values()) +
        sum(d_sltv_hc7.values()) +
        sum(d_slt_hc8.values()) +
        sum(d_sltv_hc10.values()) +
        sum(d_sltv_hc11.values()) +
        sum(d_sv_hc12.values())
    )
    
    return total_objective





In [3]:
def generate_moves(x, students, teachers, timeslots, courses, V):
    moves = []

    # Vooraf gedefinieerde actieve toewijzingen
    active_assignments = [(s, l, t, v) for (s, l, t, v), value in x.items() if value == 1]
    inactive_assignments = [(s, l, t, v) for (s, l, t, v), value in x.items() if value == 0] 

    # Insert moves – alleen voor geldige vakken (in V[s]) en waar x = 0 en andere VW'en
    for (s, l, t, v) in inactive_assignments:
        moves.append(("insert", s, l, t, v))

    # Delete moves – alleen waar x = 1
    for (s, l, t, v) in active_assignments:
        moves.append(("delete", s, l, t, v))

    # Reassignment moves
    for (s, l, t, v) in active_assignments:
        # Reassign student
        for s2 in students:
            if s != s2:
                moves.append(("reassign_student", s, s2, l, t, v))

        # Reassign teacher
        for l2 in teachers:
            if l != l2:
                moves.append(("reassign_teacher", s, l, l2, t, v))

        # Reassign timeslot
        for t2 in timeslots:
            if t != t2:
                moves.append(("reassign_timeslot", s, l, t, t2, v))

        # Reassign subject
        for v2 in V[s]:
            if v != v2:
                moves.append(("reassign_subject", s, l, t, v, v2))

    # Swap student
    for i in range(len(active_assignments)):
        for j in range(i + 1, len(active_assignments)):
            s1, l1, t1, v1 = active_assignments[i]
            s2, l2, t2, v2 = active_assignments[j]
            if (s1 != s2 and (l1 != l2 or t1 != t2 or v1 != v2)):
                moves.append(("swap_students", s1, l1, t1, v1, s2, l2, t2, v2))

    # Swap teacher
    for i in range(len(active_assignments)):
        for j in range(i + 1, len(active_assignments)):
            s1, l1, t1, v1 = active_assignments[i]
            s2, l2, t2, v2 = active_assignments[j]
            if (l1 != l2 and (s1 != s2 or t1 != t2 or v1 != v2)):
                moves.append(("swap_teachers", s1, l1, t1, v1, s2, l2, t2, v2))

    # Swap subject
    for i in range(len(active_assignments)):
        for j in range(i + 1, len(active_assignments)):
            s1, l1, t1, v1 = active_assignments[i]
            s2, l2, t2, v2 = active_assignments[j]
            if (v1 != v2 and (s1 != s2 or t1 != t2 or l1 != l2)):
                moves.append(("swap_subjects", s1, l1, t1, v1, s2, l2, t2, v2))

    # Swap timeslot
    for i in range(len(active_assignments)):
        for j in range(i + 1, len(active_assignments)):
            s1, l1, t1, v1 = active_assignments[i]
            s2, l2, t2, v2 = active_assignments[j]
            if (t1 != t2 and (s1 != s2 or l1 != l2 or v1 != v2)):
                moves.append(("swap_timeslots", s1, l1, t1, v1, s2, l2, t2, v2))

    # Swap teacher for course
    for s in students:
        for v in V[s]:
            teachers_for_v = [l for l in teachers if any(x[s, l, t, v] == 1 for t in timeslots)]
            for l1 in teachers_for_v:
                for l2 in teachers:
                    if l1 != l2:
                        for t in timeslots:
                            if x[s, l1, t, v] == 1:
                                moves.append(("swap_teacher_for_course", s, l1, l2, v))

    

    return moves




In [4]:
import copy

def apply_move(x, move):
    x_new = copy.deepcopy(x)
    move_type = move[0]

    if move_type == "insert":
        _, s, l, t, v = move
        x_new[s, l, t, v] = 1

    elif move_type == "delete":
        _, s, l, t, v = move
        x_new[s, l, t, v] = 0
        
    elif move_type == "reassign_student":
        _, s_old, s_new, l, t, v = move
        x_new[s_old, l, t, v] = 0
        x_new[s_new, l, t, v] = 1

    elif move_type == "reassign_teacher":
        _, s, l_old, l_new, t, v = move
        x_new[s, l_old, t, v] = 0
        x_new[s, l_new, t, v] = 1

    elif move_type == "reassign_timeslot":
        _, s, l, t_old, t_new, v = move
        x_new[s, l, t_old, v] = 0
        x_new[s, l, t_new, v] = 1

    elif move_type == "reassign_subject":
        _, s, l, t, v_old, v_new = move
        x_new[s, l, t, v_old] = 0
        x_new[s, l, t, v_new] = 1

    elif move_type == "swap_students":
        _, s1, l1, t1, v1, s2, l2, t2, v2 = move
        x_new[s1, l1, t1, v1], x_new[s2, l2, t2, v2] = 0, 0
        x_new[s1, l2, t2, v2], x_new[s2, l1, t1, v1] = 1, 1

    elif move_type == "swap_teachers":
        _, s1, l1, t1, v1, s2, l2, t2, v2 = move
        x_new[s1, l1, t1, v1], x_new[s2, l2, t2, v2] = 0, 0
        x_new[s1, l2, t2, v2], x_new[s2, l1, t1, v1] = 1, 1

    elif move_type == "swap_subjects":
        _, s1, l1, t1, v1, s2, l2, t2, v2 = move
        x_new[s1, l1, t1, v1], x_new[s2, l2, t2, v2] = 0, 0
        x_new[s1, l1, t1, v2], x_new[s2, l2, t2, v1] = 1, 1

    elif move_type == "swap_timeslots":
        _, s1, l1, t1, v1, s2, l2, t2, v2 = move
        x_new[s1, l1, t1, v1], x_new[s2, l2, t2, v2] = 0, 0
        x_new[s1, l1, t2, v1], x_new[s2, l2, t1, v2] = 1, 1

    elif move_type == "swap_teacher_for_course":
        _, s, l1, l2, v = move
        for t in timeslots:
            if x[s, l1, t, v] == 1:
                x_new[s, l1, t, v] = 0
                x_new[s, l2, t, v] = 1

    return x_new




In [5]:
def local_search(x_prev, students, teachers, timeslots, courses, V, R, H, m, M, T_avail, T_unavail_l, a):
    import copy

    # Start from previous schedule
    x_current = copy.deepcopy(x_prev)
    obj = calculate_total_objective(x_current, students, teachers, timeslots, courses, V, R, H, m, M, T_avail, T_unavail_l, a)

    print(f"Initial objective value: {obj}")
    
    while obj > 0:
        best_move = None
        best_delta = 0

        # Generate all possible valid moves
        moves = generate_moves(x_current, students, teachers, timeslots, courses, V)

        for move in moves:
            x_candidate = apply_move(x_current, move)
            candidate_obj = calculate_total_objective(x_candidate, students, teachers, timeslots, courses, V, R, H, m, M, T_avail, T_unavail_l, a)
            delta = obj - candidate_obj

            if delta > best_delta:
                best_delta = delta
                best_move = move

        if best_move:
            x_current = apply_move(x_current, best_move)
            obj -= best_delta
            print(f"Applied move: {best_move} | New objective: {obj}")
        else:
            print("No improving move found. Local minimum reached.")
            break

    return x_current, obj





In [6]:
def build_optimization_model(x, students, teachers, timeslots, courses, V, R, H, m, M, T_avail, T_unavail_l, a):

    y = {}
    y_slv = {}
    # Loop over all students, teachers, timeslots, and courses to compute y_prev
    for s in students:
        for l in teachers:
            for t in timeslots:
                 #Sum over all courses v that student s is assigned to at timeslot t with teacher l
                y[(s, l, t)] = sum(x[s,l,t,v] for v in V[s])
    
    #compute y_slv
    for s in students:
        for l in teachers:      
            for v in V[s]:
                y_slv[s, l, v] = int(any(x[s, l, t, v] == 1 for t in timeslots))
                
    obj_term1={}
    obj_term2={}
    obj_term3={}
    w_1 = 2/(10*sum(H[s] for s in students))
    w_2 = 1/len(timeslots)
    w_3 = 1/(2*len(timeslots))
    w_4 = 8/(2*len(timeslots))
    lambda_lt=4/(len(teachers)*len(timeslots))

    #Assignment obj_term1:
    obj_term1 = w_1*(sum(c[l, v] * x[s, l, t, v] for s in students for l in teachers for t in timeslots for v in V[s]))
    
    #Assignment obj_term2:
    z_lt = {}
    for l in teachers:
        for t in timeslots:
            overload = sum(x[s, l, t, v] for s in students for v in V[s])
            z_lt[l, t] = 1 if overload > 1 else 0
    obj_term2 = lambda_lt * sum(z_lt[l, t] for l in teachers for t in timeslots)

    #Assignment obj_term3:
    q_st={}
    Q_maxtime={}
    q_stv={}
    Q_maxsubject={}
    y_slv_prev={}
    q_slv={}
    Q_maxteacher={}
    for s in students:
        for t in timeslots:
            q_st[s,t] = abs(sum(x[s, l, t, v] for l in teachers for v in V[s]) - 
                         sum(x_prev[s, l, t, v] for l in teachers for v in V[s]))
    Q_maxtime = max(sum(q_st[s,t] for t in timeslots) for s in students)
    
    for s in students:
        for v in V[s]:
            for t in timeslots:
                q_stv[s,t,v] = abs(sum(x[s, l, t, v] for l in teachers) - sum(x_prev[s, l, t, v] for l in teachers))
    
    Q_maxsubject = max(sum(q_stv[s,t,v] for v in V[s] for t in timeslots) for s in students)
    
    for s in students:
        for l in teachers:      
            for v in V[s]:
                y_slv_prev[s, l, v] = int(any(x_prev[s, l, t, v] == 1 for t in timeslots))
    
    for s in students:
        for l in teachers:
            for v in V[s]:
                q_slv[s,l,v] = abs(y_slv[s,l,v] - y_slv_prev[s,l,v])
    
    Q_maxteacher = max(sum(q_slv[s,l,v] for l in teachers for v in V[s]) for s in students)
        
    obj_term3 = w_2*Q_maxtime + w_3*Q_maxsubject + w_4*Q_maxteacher
    
    O = obj_term1 - obj_term2 - obj_term3 
    
    Optimal_solution = O
    
    return Optimal_solution




In [ ]:
def generate_moves_opt(x, students, teachers, timeslots, courses, V):
    moves = []

    # Vooraf gedefinieerde actieve toewijzingen
    active_assignments = [(s, l, t, v) for (s, l, t, v), value in x.items() if value == 1]
    
    # Reassignment moves
    for (s, l, t, v) in active_assignments:
        # Reassign student
        for s2 in students:
            if s != s2:
                moves.append(("reassign_student", s, s2, l, t, v))

        # Reassign teacher
        for l2 in teachers:
            if l != l2:
                moves.append(("reassign_teacher", s, l, l2, t, v))

        # Reassign timeslot
        for t2 in timeslots:
            if t != t2:
                moves.append(("reassign_timeslot", s, l, t, t2, v))

        # Reassign subject
        for v2 in V[s]:
            if v != v2:
                moves.append(("reassign_subject", s, l, t, v, v2))

    # Swap student
    for i in range(len(active_assignments)):
        for j in range(i + 1, len(active_assignments)):
            s1, l1, t1, v1 = active_assignments[i]
            s2, l2, t2, v2 = active_assignments[j]
            if (s1 != s2 and (l1 != l2 or t1 != t2 or v1 != v2)):
                moves.append(("swap_students", s1, l1, t1, v1, s2, l2, t2, v2))

    # Swap teacher
    for i in range(len(active_assignments)):
        for j in range(i + 1, len(active_assignments)):
            s1, l1, t1, v1 = active_assignments[i]
            s2, l2, t2, v2 = active_assignments[j]
            if (l1 != l2 and (s1 != s2 or t1 != t2 or v1 != v2)):
                moves.append(("swap_teachers", s1, l1, t1, v1, s2, l2, t2, v2))

    # Swap subject
    for i in range(len(active_assignments)):
        for j in range(i + 1, len(active_assignments)):
            s1, l1, t1, v1 = active_assignments[i]
            s2, l2, t2, v2 = active_assignments[j]
            if (v1 != v2 and (s1 != s2 or t1 != t2 or l1 != l2)):
                moves.append(("swap_subjects", s1, l1, t1, v1, s2, l2, t2, v2))

    # Swap timeslot
    for i in range(len(active_assignments)):
        for j in range(i + 1, len(active_assignments)):
            s1, l1, t1, v1 = active_assignments[i]
            s2, l2, t2, v2 = active_assignments[j]
            if (t1 != t2 and (s1 != s2 or l1 != l2 or v1 != v2)):
                moves.append(("swap_timeslots", s1, l1, t1, v1, s2, l2, t2, v2))

    # Swap teacher for course
    for s in students:
        for v in V[s]:
            teachers_for_v = [l for l in teachers if any(x[s, l, t, v] == 1 for t in timeslots)]
            for l1 in teachers_for_v:
                for l2 in teachers:
                    if l1 != l2:
                        for t in timeslots:
                            if x[s, l1, t, v] == 1:
                                moves.append(("swap_teacher_for_course", s, l1, l2, v))

    

    return moves




In [7]:
def local_search_opt(x_start):
    x_current = copy.deepcopy(x_start)
    obj = build_optimization_model(x_current, students, teachers, timeslots, courses, V, R, H, m, M, T_avail, T_unavail_l, a)

    print(f"Initial Objective: {obj}")
    
    best_obj = obj
    best_solution = copy.deepcopy(x_current)

    while True:
        best_move = None
        best_delta = 0
        N = generate_moves_opt(x_current, students, teachers, timeslots, courses, V)
        for move in N:
            x_candidate = apply_move(x_current, move)

            # Constraint 1: student can only be assigned during their available times
            if any(x_candidate[s, l, t, v] == 1 
                   for s in students
                   for l in teachers
                   for v in V[s]
                   for t in timeslots
                   if t not in T_avail[s]):
                continue

            # Constraint 2: teacher must not be scheduled during their unavailable times
            if any(x_candidate[s, l, t, v] == 1 
                   for s in students
                   for l in teachers
                   for t in T_unavail_l.get(l, set())
                   for v in V[s]):
                continue

            # Constraint 3: one course max per student per time
            if any(sum(x_candidate[s, l, t, v] for l in teachers for v in V[s]) > 1 
                   for s in students 
                   for t in T_avail[s]):
                continue

            # Constraint 4: total hours assigned to student == H[s]
            if any(sum(x_candidate[s, l, t, v] for l in teachers for t in timeslots for v in V[s]) != H[s]
                   for s in students):
                continue

            # Constraint 5: one teacher per student-course
            if any(sum(1 for l in teachers if sum(x_candidate[s, l, t, v] for t in timeslots) > 0) != 1
                   for s in students
                   for v in V[s]):
                continue

            # Constraint 6: subject hours within [m, M]
            if any(
                (sum(x_candidate[s, l, t, v] for l in teachers for t in timeslots) < m[s, v]) or
                (sum(x_candidate[s, l, t, v] for l in teachers for t in timeslots) > M[s, v])
                for s in students for v in V[s]
            ):
                continue

            # Constraint 7: respect relationship factor R
            if any(x_candidate[s, l, t, v] == 1 and R[s][l] == 0
                   for s in students
                   for l in teachers
                   for v in V[s]
                   for t in timeslots):
                continue

            # Constraint 8: NIEUWE VERSIE
            if any(
                sum(x_candidate[s_prime, l, t, v_prime] 
                    for s_prime in students if s_prime != s 
                    for v_prime in V[s_prime]) > (1 - a[s]) + (1 - sum(x_candidate[s, l, t, v] for v in V[s])) * 2
                for s in students
                for l in teachers
                for t in timeslots):
                continue

             # Constraint 9: student must not be assigned to a course outside their curriculum
            if any(x_candidate[s, l, t, v] == 1
                   for s in students
                   for l in teachers
                   for t in timeslots
                   for v in courses
                   if v not in V[s]):
                continue
   
            obj_candidate = build_optimization_model(x_candidate, students, teachers, timeslots, courses, V, R, H, m, M, T_avail, T_unavail_l, a)
            delta = obj_candidate - obj  #reversed

            if delta > 0:
                if delta > best_delta:
                    best_delta = delta
                    best_move = move


        if best_move:
            x_current = apply_move(x_current, best_move)
            obj += best_delta
            print(f"Applied move: {best_move} | New objective: {obj}")
            # Update best solution if this is the best so far
            if obj > best_obj:
                best_obj = obj
                best_solution = copy.deepcopy(x_current)
        else:
            print("No improving move found. Local minimum reached.")
            break

    return x_current, obj





In [8]:


x_final, final_obj = local_search(
    x_prev, students, teachers, timeslots, courses, V, R, H, m, M, T_avail, T_unavail_l, a
)

print("Feasibility phase search complete. Final objective:", final_obj)

x_optimal, optimal_obj = local_search_opt(x_final)

end_time = time.time()
elapsed_time = end_time - start_time

print("Search complete. Final objective:", optimal_obj)
print(f"Total running time: {elapsed_time:.4f} seconds")




Initial objective value: 107.0


KeyboardInterrupt: 